In [1]:
import torch
print(torch.cuda.is_available())  # Should print True if CUDA is available
print(torch.cuda.current_device())

True
0


In [2]:
torch.cuda.empty_cache()

In [1]:
!nvidia-smi

Mon Oct 21 20:24:12 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     On  |   00000000:81:00.0 Off |                  N/A |
| 20%   32C    P8              9W /  250W |       2MiB /  11264MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [12]:
!kill -9 2710969                                                                                                                                                

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

2024-10-21 20:24:18.068229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 20:24:18.087654: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-21 20:24:18.093628: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-21 20:24:18.108208: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-21 20:24:19.201312: W tensorflow/compiler/tf2

In [3]:
import time

In [4]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)

/upb/users/b/balram/profiles/unix/cs/.local/lib/python3.12/site-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [5]:
# Load the model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,  # Enable 4-bit quantization
    device_map="auto",  # Automatically place model layers on GPU
    torch_dtype=torch.float16
    #quantization_config=bnb.BnbQuantizationConfig(load_in_4bit=True)
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
# Create the text generation pipeline
text_generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [8]:
import json
import jsonlines
import re

In [9]:
def load_prompts_data(filepath):
    with jsonlines.open(filepath) as reader:
        return list(reader)

In [10]:
def extract_prompt_info(item):
    prompt_id = item.get('id')
    prompt_text = item.get('prompt')
    return prompt_id, prompt_text

In [11]:
# Assuming 'text_generator' is already defined somewhere in your environment
def generate_text(prompt_text, max_length=1000, max_new_tokens=300,num_return_sequences=1):
    response = text_generator(prompt_text, max_length=max_length, num_return_sequences=num_return_sequences)
    return response

In [12]:
def extract_test_output(response):
    if response and len(response) > 0:
        generated_text = response[0].get('generated_text', '')
        match = re.search(r'Test Output:\s*(.*?)\s*(?:\n|$)', generated_text, re.DOTALL)
        if match:
            return match.group(1).strip()
    return 'Output not found'

In [13]:
def parse_model_output(model_output):
    # Initialize an empty list to store triples
    triples = []
    
    # Remove leading/trailing whitespace and split the input data into lines
    lines = [line.strip() for line in model_output.strip().split('\n')]
    
    # Define the regex pattern to match triples in the format: relation(subject, object)
    pattern = re.compile(r'(.+?)\s*\(([^,]+),\s*([^)]+)\)')
    
    for line in lines:
        # Find all matches for the pattern in the line
        matches = pattern.findall(line)
        
        for match in matches:
            relation, subject, obj = match
            # Clean up subject and object values
            subject = subject.strip()
            obj = obj.strip()
            
            # Append the extracted triple to the list
            triples.append({"sub": subject, "rel": relation, "obj": obj})
    
    # Return the list of triples
    return triples

In [14]:
#JSONL_FILEPATH = 'ont_1_movie_prompts.jsonl'
#output_filepath='LLM_Response.jsonl'


#JSONL_FILEPATH = 'dbpedia_webnig/Input_Prompts/ont_19_film_prompts.jsonl'
#output_filepath='dbpedia_webnig/Response/ont_19_film_llm_response.jsonl'

JSONL_FILEPATH = 'dbpedia_webnig/Input_Prompts/ont_16_city_prompts.jsonl'
output_filepath='dbpedia_webnig/Response/ont_16_city_llm_response.jsonl'

In [15]:
import json

def save_triples_to_jsonl(processed_data, output_filepath):
    """
    Save the processed triples into a JSONL file format.

    Args:
        processed_data (list): A list of dictionaries containing the 'id' and 'triples'.
        output_filepath (str): The output file path where the JSONL data should be saved.
    """
    with open(output_filepath, 'w', encoding='utf-8') as outfile:
        for entry in processed_data:
            jsonl_entry = {
                "id": entry["id"],
                "triples": entry["triples"]
            }
            # Convert dictionary to a JSON string and write it to the file
            outfile.write(json.dumps(jsonl_entry, ensure_ascii=False) + "\n")

In [16]:
def main(filepath, output_filepath, num_prompts=4):
    prompts_data = load_prompts_data(filepath)
    processed_data = []
    for i in range(min(num_prompts, len(prompts_data))):
        item = prompts_data[i]
        prompt_id, prompt_text = extract_prompt_info(item)
        response = generate_text(prompt_text)
        test_output = extract_test_output(response)
        
        # Debugging: Print the test output to ensure it's correctly extracted
        print(f"Test Output for ID {prompt_id}: {test_output}")
        
        # Parse the test output into triples
        triples = parse_model_output(test_output)
        
        # Debugging: Print the parsed triples to ensure they're correct
        print(f"Parsed Triples for ID {prompt_id}: {triples}")
        
        processed_entry = {
            "id": prompt_id,
            "triples": triples
        }
        
        processed_data.append(processed_entry)
    
    # Save the processed data into a new jsonl file
    save_triples_to_jsonl(processed_data, output_filepath)
    print(f"Processed triples saved to {output_filepath}")

In [17]:
# Replace 'your_input_filepath.jsonl' with your actual input file path
# Replace 'your_output_filepath.jsonl' with your desired output file path
main(JSONL_FILEPATH, output_filepath, num_prompts=217)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/upb/users/b/balram/profiles/unix/cs/.local/lib/python3.12/site-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Test Output for ID ont_16_city_test_1: country(Albany, United States)
Parsed Triples for ID ont_16_city_test_1: [{'sub': 'Albany', 'rel': 'country', 'obj': 'United States'}]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Test Output for ID ont_16_city_test_2: populationDensity(Albany, Georgia, 535 (inhabitants per square kilometer))
Parsed Triples for ID ont_16_city_test_2: [{'sub': 'Albany', 'rel': 'populationDensity', 'obj': 'Georgia, 535 (inhabitants per square kilometer'}]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Test Output for ID ont_16_city_test_214: ethnicGroup(United States, Asian Americans)
Parsed Triples for ID ont_16_city_test_214: [{'sub': 'United States', 'rel': 'ethnicGroup', 'obj': 'Asian Americans'}]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Test Output for ID ont_16_city_test_215: (Note: In this case, there is no specific city mentioned, but we can still infer the triples based on the context.)
Parsed Triples for ID ont_16_city_test_215: []


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Test Output for ID ont_16_city_test_216: (Note: The Demonym for Indiana is Hoosier)
Parsed Triples for ID ont_16_city_test_216: []
Test Output for ID ont_16_city_test_217: (Note: The test sentence has more information than the example sentence, so the output will have more triples.)
Parsed Triples for ID ont_16_city_test_217: []
Processed triples saved to dbpedia_webnig/Response/ont_16_city_llm_response.jsonl


In [ ]:




.

In [ ]:
RTC:dbpedia_webnig/Response/Mistral_Response/ont_1_university_llm_response.jsonl